In [21]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import random
import copy
import math
import matplotlib.pyplot as plt
import argparse
from collections import OrderedDict
from tqdm import tqdm
from preprocess_dataset import ProcessedDataLoader

import subprocess
from train import load_cifar10_data, validate, make_rank_k_model
device = "cpu"


In [22]:
def eval_model(model, model_name, data_aug=1):
    train_dataset = load_cifar10_data('train', './datasets',1)
    val_dataset = load_cifar10_data('val', './datasets',1)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=1, pin_memory=True)

    criterion = nn.CrossEntropyLoss().to(device)

    train_acc, train_loss = validate(model, device, train_loader, criterion)
    val_acc, val_loss = validate(model, device, val_loader, criterion)

    print(f'=================== Summary for model {model_name}===================\n',
          f'Training loss: {train_loss:.3f}   Validation loss {val_loss:.3f}   ',
          f'Training accuracy: {train_acc:.3f}   Validation accuracy: {val_acc:.3f}\n')

    return train_acc, val_acc, train_loss, val_loss

In [23]:
import os
import torch

# Define the root directory where your models are stored
# make an array with 4 file paths; collect val acc and values for k, save in an array
root_dir = '/home/andrew/andrew/89_FP/cs89finalproject/models/no_regularization/data_aug/nunits=256'

# Walk through the directory
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pt'):
            # Construct the full path to the file
            file_path = os.path.join(dirpath, filename)
            model_obj = torch.load(file_path, map_location=torch.device('cpu'))

            # for the hps that were saved in a dumb way
            weights = model_obj['model state']
            congif = model_obj['hps']
            
            nchannels = congif.nchannels
            nunits = congif.nunits
            nclasses = congif.nclasses
            nlayers = congif.nlayers
            rank_constraint = congif.rank_constraint


            # rank_constraint = model_obj['args']['rank_constraint']

            model = make_rank_k_model(nchannels=nchannels, nunits=nunits, nclasses=nclasses, nlayers=nlayers, k=rank_constraint)
            model.load_state_dict(weights)

            train_acc, val_acc, train_loss, val_loss = eval_model(model, f"Model with each weight layer having rank at most {rank_constraint} during training")
            




{'model state': OrderedDict({'fc1_down.weight': tensor([[-0.0801, -0.0123,  0.0229,  ...,  0.1426,  0.1592,  0.1389],
        [-0.0350, -0.0661, -0.1151,  ...,  0.0476,  0.0928,  0.0626],
        [-0.0253, -0.0283, -0.0183,  ...,  0.0032, -0.0317, -0.0888],
        ...,
        [ 0.2005,  0.1472,  0.1107,  ...,  0.0406,  0.0618,  0.0875],
        [ 0.0548,  0.0485,  0.0052,  ...,  0.0630,  0.0991,  0.1369],
        [-0.0683, -0.1038, -0.1145,  ...,  0.0563,  0.0685,  0.0558]]), 'fc1_down.bias': tensor([ 0.0540,  0.1080, -0.2863, -0.4344, -0.1076, -0.4822, -0.1582, -0.3843,
         0.3848]), 'fc1_up.weight': tensor([[-2.9545e-02, -3.3148e-01,  9.4741e-02,  ...,  3.7533e-02,
          4.0661e-02, -6.4947e-02],
        [ 3.9053e-03, -1.0990e-01,  3.1561e-02,  ..., -1.9454e-01,
          1.4878e-01, -1.7816e-02],
        [-3.3945e-03,  9.3010e-03, -7.8062e-02,  ...,  1.7242e-01,
         -1.2539e-01,  1.3190e-01],
        ...,
        [-1.3697e-01,  4.1996e-03, -8.9027e-02,  ...,  1.1725e